In [ ]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm

CASE_URL = "https://api.vitaldb.net/cases"
TRACK_URL = "https://api.vitaldb.net/trks"
DATA_URL = "https://api.vitaldb.net/{}"

PARAMETERS = {
    'Solar8000/HR': 'Heart Rate',
    'Solar8000/ART_MBP': 'Arterial BP',
    'Solar8000/PLETH_SPO2': 'Oxygen Saturation',
    'Solar8000/ETCO2': 'End-Tidal CO2',
    'Solar8000/RR': 'Respiratory Rate',
    'Solar8000/PLETH_HR': 'Pleth HR',
    'Solar8000/ART_SBP': 'Systolic BP',
    'Solar8000/ART_DBP': 'Diastolic BP',
    'Solar8000/NIBP_MBP': 'NIBP Mean BP',
    'Solar8000/CVP': 'Central Venous Pressure',
    'Vigileo/CO': 'Cardiac Output',
    'Vigileo/SV': 'Stroke Volume',
    'Vigilance/HR_AVG': 'Average HR',
    'CardioQ/HR': 'CardioQ HR'
}

cases_df = pd.read_csv(CASE_URL)
all_caseids = cases_df['caseid'].unique().tolist()[:500]  # Limit to 500 cases


tracks_df = pd.read_csv(TRACK_URL)
filtered_tracks = tracks_df[
    (tracks_df['caseid'].isin(all_caseids)) &
    (tracks_df['tname'].isin(PARAMETERS.keys()))
]

grouped = filtered_tracks.groupby('caseid')

results = []

for caseid, group in tqdm(grouped, desc="Analyzing Correlations"):
    dfs = {}
    for _, row in group.iterrows():
        tname = row['tname']
        label = PARAMETERS[tname]

        df = pd.read_csv(DATA_URL.format(row['tid']), header=None, names=['time', 'value'])
        df = df.dropna()
        df['time'] = pd.to_numeric(df['time'], errors='coerce')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        df = df.dropna()
        df['bin'] = (df['time'] // 900).astype(int)
        df = df.groupby('bin')['value'].mean().reset_index()
        dfs[label] = df.set_index('bin')['value']


    if 'Heart Rate' not in dfs or len(dfs) < 2:
        continue

    merged_df = pd.DataFrame(dfs).dropna()
    if merged_df.shape[0] < 2:
        continue

    corrs = merged_df.corr()['Heart Rate'].drop('Heart Rate', errors='ignore')
    if corrs.empty:
        continue

    strongest = corrs.abs().max()
    results.append((caseid, strongest))

#  Return top 20 strongest correlations 
top20 = sorted(results, key=lambda x: x[1], reverse=True)[:50]
top20_caseids = [caseid for caseid, _ in top20]
print("Top 20 correlated case IDs:", top20_caseids)


Analyzing Correlations: 100%|██████████| 500/500 [04:13<00:00,  1.97it/s]

Top 20 correlated case IDs: [59, 353, 163, 421, 279, 58, 183, 145, 221, 419, 425, 460, 92, 242, 384, 405, 413, 427, 96, 380, 60, 297, 375, 416, 435, 68, 215, 385, 458, 229, 150, 116, 139, 256, 239, 266, 203, 25, 161, 175, 135, 142, 323, 291, 137, 314, 243, 478, 2, 477]


In [1]:
import pandas as pd
import json
import os
from tqdm import tqdm

PARAMETERS = {
    'Solar8000/HR': 'Heart Rate',
    'Solar8000/ART_MBP': 'Arterial BP',
    'Solar8000/PLETH_SPO2': 'Oxygen Saturation',
    'Solar8000/ETCO2': 'End-Tidal CO2',
    'Solar8000/RR': 'Respiratory Rate',
    'Solar8000/PLETH_HR': 'Pleth HR',
    'Solar8000/ART_SBP': 'Systolic BP',
    'Solar8000/ART_DBP': 'Diastolic BP',
    'Solar8000/NIBP_MBP': 'NIBP Mean BP',
    'Solar8000/CVP': 'Central Venous Pressure',
    'Vigileo/CO': 'Cardiac Output',
    'Vigileo/SV': 'Stroke Volume',
    'Vigilance/HR_AVG': 'Average HR',
    'CardioQ/HR': 'CardioQ HR'
}

CASE_IDS = [25, 58, 68, 92, 96, 116, 135, 137, 142, 161, 203, 229, 239,
            243, 256, 266, 279, 323, 375, 384, 413, 416, 458, 460, 478]

TRKS_CSV_URL = 'https://api.vitaldb.net/trks'
TRACK_DATA_URL = 'https://api.vitaldb.net/{}.csv'


def interpolate_per_second(df):
    df['sec'] = df['time'].astype(int)
    df = df.drop_duplicates(subset='sec', keep='last')
    return df[['sec', 'value']].rename(columns={'sec': 'time'}).to_dict(orient='records')


print("📥 Downloading track metadata...")
tracks_df = pd.read_csv(TRKS_CSV_URL)
filtered_tracks = tracks_df[
    (tracks_df['caseid'].isin(CASE_IDS)) &
    (tracks_df['tname'].isin(PARAMETERS.keys()))
]

result = {}
for case_id in tqdm(CASE_IDS, desc="Cases"):
    case_tracks = filtered_tracks[filtered_tracks['caseid'] == case_id]
    if case_tracks.empty:
        print(f"[Case {case_id}] No matching parameters. Skipping.")
        continue

    case_data = {}
    for _, row in case_tracks.iterrows():
        label = PARAMETERS[row['tname']]
        tid = row['tid']
        try:
            df = pd.read_csv(TRACK_DATA_URL.format(tid), names=['time', 'value'], header=None)
            df.dropna(inplace=True)
            df['time'] = pd.to_numeric(df['time'], errors='coerce')
            df['value'] = pd.to_numeric(df['value'], errors='coerce')
            df.dropna(inplace=True)
            values = interpolate_per_second(df)
            case_data[label] = values
        except Exception as e:
            print(f"[Case {case_id} - {label}] Error loading tid {tid}: {e}")

    result[str(case_id)] = case_data

with open('vital_data.json', 'w') as f:
    json.dump(result, f, indent=2)

print("✅ vital_data.json saved.")



📥 Downloading track metadata...


Cases:   0%|          | 0/25 [00:00<?, ?it/s]

[Case 25 - Diastolic BP] Error loading tid 6b85159a14baf492708289e3a20a9f7c29108533: HTTP Error 403: Forbidden
[Case 25 - Arterial BP] Error loading tid 86e7ee91a60039136ed5c3c64ce6e9a2052495be: HTTP Error 403: Forbidden
[Case 25 - Systolic BP] Error loading tid a5f6c3e020f7d8a9c7f2ffe116c3257a32ff95fb: HTTP Error 403: Forbidden
[Case 25 - End-Tidal CO2] Error loading tid d17bd79d8f74f318410dbf8018034a2aa6dff015: HTTP Error 403: Forbidden
[Case 25 - Heart Rate] Error loading tid c5ffe2448269dc898f31e25c2ad3a1b6c79f62a0: HTTP Error 403: Forbidden
[Case 25 - NIBP Mean BP] Error loading tid 3deb3575de30da54df55c9b502454466cd4788d1: HTTP Error 403: Forbidden
[Case 25 - Pleth HR] Error loading tid 9d17636f4a7047e86cf7fb95356925d829ff5ef2: HTTP Error 403: Forbidden


Cases:   4%|▍         | 1/25 [00:05<02:00,  5.03s/it]

[Case 25 - Oxygen Saturation] Error loading tid 16ceb678a45e36a173b621792df2973e4918a41d: HTTP Error 403: Forbidden
[Case 58 - Diastolic BP] Error loading tid ffa1843212257085240aa41b92d4fdc49e290656: HTTP Error 403: Forbidden
[Case 58 - Arterial BP] Error loading tid 7ddfb5fa2e94f725e347fc4d2455901d2b2030ef: HTTP Error 403: Forbidden
[Case 58 - Systolic BP] Error loading tid 42e03641eb91128c7d2309761338f18b4981d5f1: HTTP Error 403: Forbidden
[Case 58 - Central Venous Pressure] Error loading tid 347ec34242d9b1a17a6edb876fbc7cbebf3a4d9a: HTTP Error 403: Forbidden
[Case 58 - End-Tidal CO2] Error loading tid 53e967993820d9efb9070959191b79f744c4cdb7: HTTP Error 403: Forbidden
[Case 58 - Heart Rate] Error loading tid 312e87b2d3866c888dc1687ff64c51750186eb06: HTTP Error 403: Forbidden
[Case 58 - NIBP Mean BP] Error loading tid 7422de2319c8ad219645400d83ed83f54bc0a335: HTTP Error 403: Forbidden
[Case 58 - Pleth HR] Error loading tid b0a5b6d608a6f9b2d8bd6b83da6a968196e52bf6: HTTP Error 403: Fo

...

[Case 458 - Respiratory Rate] Error loading tid 91b1bca6d13280795015d7e5288fc1d0ff88c20f: HTTP Error 403: Forbidden
[Case 460 - Diastolic BP] Error loading tid c830de303c3f81e7de3235d776fbd428409eaaec: HTTP Error 403: Forbidden
[Case 460 - Arterial BP] Error loading tid c6fe9c362ca66d4ca47437368b82bf552e17ff16: HTTP Error 403: Forbidden
[Case 460 - Systolic BP] Error loading tid 98bcbada1b2427ca48f3ddff08ec3873036197ee: HTTP Error 403: Forbidden
[Case 460 - Central Venous Pressure] Error loading tid 6f5c4be7057cf42ca08596e7d9c1730981045543: HTTP Error 403: Forbidden
[Case 460 - End-Tidal CO2] Error loading tid 3223a5a024196489aa30086531fc4ac4b262ba26: HTTP Error 403: Forbidden
[Case 460 - Heart Rate] Error loading tid 585d2bd60ae44cdeb14a752e71260e88be7b8a61: HTTP Error 403: Forbidden
[Case 460 - NIBP Mean BP] Error loading tid 4fcd7f59b549e56a999c98609c864bc30816d61a: HTTP Error 403: Forbidden
[Case 460 - Pleth HR] Error loading tid 513fe35864a91b26168c57faa132d3793325bbb2: HTTP Error

Cases:  96%|█████████▌| 24/25 [02:04<00:05,  5.21s/it]

[Case 460 - Oxygen Saturation] Error loading tid 4584eea7c4b74092c96c0e93b36a3e09671d11c0: HTTP Error 403: Forbidden
[Case 478 - Diastolic BP] Error loading tid fb8b72a74e19a30c6d2e770ed9b516211e9d456c: HTTP Error 403: Forbidden
[Case 478 - Arterial BP] Error loading tid 5fea2b7fe296e8ea0b2f4fbc7530933fd31d6d13: HTTP Error 403: Forbidden
[Case 478 - Systolic BP] Error loading tid d45282160ef1608aa2c0b966fc3083fc48317c00: HTTP Error 403: Forbidden
[Case 478 - End-Tidal CO2] Error loading tid 69d58f40669b06b63769242d5cffafe3243375f4: HTTP Error 403: Forbidden
[Case 478 - Heart Rate] Error loading tid 131ac9df3b2f61412994ab5002ebac68d84748f8: HTTP Error 403: Forbidden
[Case 478 - NIBP Mean BP] Error loading tid 779a6c364da9a14d4a7dde24d952d2a6128a31d8: HTTP Error 403: Forbidden
[Case 478 - Pleth HR] Error loading tid 95ea8a0b04e7d07c215c07d2a9f3f70f61e46b5c: HTTP Error 403: Forbidden


Cases: 100%|██████████| 25/25 [02:09<00:00,  5.18s/it]

[Case 478 - Oxygen Saturation] Error loading tid 10d4148ccf6e3f1c1d30b98f4ce3dd62736759bc: HTTP Error 403: Forbidden
✅ vital_data.json saved.
